### Importing libraries

In [22]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

### Importing Dataset

In [23]:
# Locating file in directories rather exposing the paths

import os
def search_file(start_dir, target_file):
    for dirpath, dirnames, filenames in os.walk(start_dir):
        if target_file in filenames:
            #print("Found {} at: {}".format(target_file, os.path.join(dirpath, target_file)))
            return os.path.join(dirpath, target_file)
    #print("File '{}' not found in '{}'".format(target_file, start_dir))
    return None

start_dir = "D:\\" 

Fault_free_data =  search_file(start_dir, "TEP_FaultFree_Training.RData")
Fault_data =  search_file(start_dir, "TEP_Faulty_Training.RData")

In [24]:
df_FaultFree = pyreadr.read_r(Fault_free_data)['fault_free_training']
df_Faulty = pyreadr.read_r(Fault_data)['faulty_training']

In [25]:
DF = pd.concat([df_FaultFree,df_Faulty]) # Join both fault free and faulty dataset
DF.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [26]:
reduced_data = DF[DF['simulationRun']==1]

#reduced fault 3, 9 and 15 due to extremely low std between fault free and faulty data
reduced_data = reduced_data[reduced_data['faultNumber'] != 3]
reduced_data = reduced_data[reduced_data['faultNumber'] != 9]
reduced_data = reduced_data[reduced_data['faultNumber'] != 15]

reduced_data

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,20.0,1.0,496,0.20006,3708.5,4497.7,9.2167,27.103,42.336,2702.7,...,53.749,20.049,59.346,15.976,38.463,42.294,44.981,48.511,41.290,21.141
9996,20.0,1.0,497,0.31328,3666.5,4483.6,9.2536,26.704,42.172,2703.3,...,53.607,30.819,60.733,15.283,38.340,38.185,46.823,48.597,41.941,18.724
9997,20.0,1.0,498,0.31186,3674.1,4480.6,9.2912,26.986,42.337,2704.0,...,53.642,30.958,62.280,14.792,38.143,32.580,45.048,48.664,41.112,16.435
9998,20.0,1.0,499,0.30026,3690.1,4519.3,9.2301,26.858,42.376,2704.2,...,53.440,29.814,59.285,14.388,39.685,36.612,43.100,48.896,41.682,18.685
